# Loading Responses

In [14]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("gaze-analysis/result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "Qwen2.5-VL-72B-Instruct" & df$list_id != -1 & df$Part != "p0", ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))

Rows: 155393 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Fitting models

In [15]:
model0 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.0074527 (tol = 0.002, component 1)”


In [16]:
model0

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Actor:Candidates) + (1 | Stimulus_ID) +  
    (1 | Prompt_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 6573.577  6637.521 -3277.788  6555.577      8991 
Random effects:
 Groups           Name        Std.Dev. 
 Stimulus_ID      (Intercept) 4.4111622
 Actor:Candidates (Intercept) 0.0006378
 Prompt_ID        (Intercept) 0.1890927
Number of obs: 9000, groups:  
Stimulus_ID, 900; Actor:Candidates, 18; Prompt_ID, 12
Fixed Effects:
                       (Intercept)                           Angleleft  
                          -0.05865                             0.59431  
                        Angleright     scale(Proximity, scale = FALSE)  
                           0.44301                            -0.30360 

In [17]:
model1 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

In [18]:
model2 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00429741 (tol = 0.002, component 1)”


In [19]:
model3 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

# Model selection

In [20]:
anova(model1, model3)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model3,7,6579.408,6629.143,-3282.704,6565.408,NA,NA,NA
model1,8,6571.577,6628.416,-3277.788,6555.577,9.83181,1,0.001715195


In [21]:
model <- model1

# Interpretation

In [22]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID) + (1 | Prompt_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  6571.6   6628.4  -3277.8   6555.6     8992 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.3560 -0.1653 -0.0912  0.1525  3.2989 

Random effects:
 Groups      Name        Variance Std.Dev.
 Stimulus_ID (Intercept) 19.45847 4.4112  
 Prompt_ID   (Intercept)  0.03576 0.1891  
Number of obs: 9000, groups:  Stimulus_ID, 900; Prompt_ID, 12

Fixed effects:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                        -0.05875    0.34654  -0.170   0.8654    
Angleleft                           0.59420    0.42855   1.387   0.1656    
Angleright                          0.44299    0.421

In [23]:
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

term,Estimate,Std. Error,z value,Pr(>|z|),effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-0.05874998,0.3465383,-0.1695339,8.653767e-01,4.773269,-0.01230812
Angleleft,0.59419945,0.4285504,1.3865335,1.655841e-01,4.773269,0.12448480
Angleright,0.44299257,0.4216683,1.0505711,2.934556e-01,4.773269,0.09280695
"scale(Proximity, scale = FALSE)",-0.30350333,0.2108875,-1.4391721,1.501018e-01,4.773269,-0.06358395
"scale(n_candidates, scale = FALSE)",-1.14493114,0.2329793,-4.9143050,8.909797e-07,4.773269,-0.23986310
ActorY,-0.74397804,0.3493456,-2.1296331,3.320191e-02,4.773269,-0.15586341


In [24]:
avg_predictions(model, by="Angle", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(left) - (front),0.045212145,0.03257338,1.38800902,0.1651343,2.59828829,-0.01863051,0.10905480
(right) - (front),0.046436512,0.03692469,1.25760070,0.2085362,2.26163026,-0.02593454,0.11880757
(right) - (left),0.001224367,0.03852652,0.03177984,0.9746476,0.03704737,-0.07428623,0.07673496


In [25]:
avg_slopes(model, variables ="Proximity", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Proximity,dY/dX,-0.3035033,0.2108936,-1.43913,0.1501137,2.735873,-0.7168472,0.1098406,-1.289874,-1.289935,-1.289904


In [26]:
avg_slopes(model, variables ="n_candidates", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
n_candidates,dY/dX,-1.144931,0.2329742,-4.914411,8.904982e-07,20.09888,-1.601552,-0.68831,-1.28979,-1.290019,-1.289904


In [27]:
avg_predictions(model, by="Actor", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Y) - (X),-0.06763862,0.02963053,-2.282734,0.02244605,5.477394,-0.1257134,-0.009563842
